In [4]:
import geopandas as gpd
import fiona; 

{'DXF': 'rw',
 'CSV': 'raw',
 'OpenFileGDB': 'raw',
 'ESRIJSON': 'r',
 'ESRI Shapefile': 'raw',
 'FlatGeobuf': 'raw',
 'GeoJSON': 'raw',
 'GeoJSONSeq': 'raw',
 'GPKG': 'raw',
 'GML': 'rw',
 'OGR_GMT': 'rw',
 'GPX': 'rw',
 'Idrisi': 'r',
 'MapInfo File': 'raw',
 'DGN': 'raw',
 'PCIDSK': 'raw',
 'OGR_PDS': 'r',
 'S57': 'r',
 'SQLite': 'raw',
 'TopoJSON': 'r'}

In [5]:
gdf_all = gpd.read_file('../Data\cbs\wijkenbuurten_2023.gpkg')
gdf_wijken = gpd.read_file('../Data/cbs/wijken/wijken_2023_v1.shp')

In [7]:
# get wijknamen from wijkcodes
def get_wijknaam(wijkcode):
    return gdf_wijken.loc[gdf_wijken['WK_CODE'] == wijkcode, 'WK_NAAM'].values[0]

gdf_all['wijknaam'] = gdf_all['wijkcode'].apply(get_wijknaam)


# reorder columns so that wijknaam is after wijkcode
cols_all = gdf_all.columns.tolist()
i_wijknaam = cols_all.index('wijknaam')
i_wijkcode = cols_all.index('wijkcode')
cols_all = cols_all[0:i_wijkcode + 1] + ['wijknaam'] + cols_all[i_wijkcode + 1:i_wijknaam]

gdf_all = gdf_all[cols_all]

# drop all rows where water is JA
gdf_all = gdf_all[gdf_all['water'] == 'NEE']

# drop rows where aantal_inwoners is 0 or -99999999.0
gdf_all = gdf_all[gdf_all['aantal_inwoners'] > 0]


irrelevant_columns = ['indelingswijziging_wijken_en_buurten', 'water', 'meest_voorkomende_postcode', 'dekkingspercentage', 'jrstatcode', 'jaar']

# drop irrelevant columns
gdf_all = gdf_all.drop(columns=irrelevant_columns)

gdf_all.reset_index(drop=True, inplace=True)

In [11]:
gdf_all.to_file('Data/cbs/wijkenbuurten_2023_clean.gpkg', driver='GPKG')